# Image Captioning - Simplified & Optimized Version

This notebook consolidates the original code by:
- **Combining redundant functions** (e.g., multiple text processing steps into one)
- **Removing unnecessary intermediate steps**
- **Using built-in methods** where possible
- **Streamlining the workflow** from ~20 functions to ~8 core functions

---

## Complete Workflow (Simplified)

```
1. SETUP & DATA PREP
   └─ process_captions() - Load, parse, clean, and save all at once

2. FEATURE EXTRACTION
   └─ extract_all_features() - Extract and save CNN features

3. PREPARE TRAINING DATA
   └─ prepare_dataset() - Load train split + create tokenizer

4. BUILD MODEL
   └─ build_model() - Define architecture

5. TRAIN
   └─ train_generator() - Create generator and train

6. INFERENCE
   └─ predict_caption() - Generate caption for new image
```

---

## 1. Imports

In [ ]:
import os
import string
import numpy as np
from PIL import Image
from pickle import dump, load
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

## 2. Configuration

In [ ]:
# Update these paths to your dataset location
DATASET_TEXT = "path/to/Flickr8k_text"
DATASET_IMAGES = "path/to/Flicker8k_Dataset"

# Files
TOKEN_FILE = os.path.join(DATASET_TEXT, "Flickr8k.token.txt")
TRAIN_IMAGES = os.path.join(DATASET_TEXT, "Flickr_8k.trainImages.txt")
TEST_IMAGES = os.path.join(DATASET_TEXT, "Flickr_8k.testImages.txt")

# Output files
FEATURES_FILE = "features.pkl"
TOKENIZER_FILE = "tokenizer.pkl"
MODEL_DIR = "models"

## 3. Data Preparation - ALL IN ONE FUNCTION

**Original code had 5 separate functions:**
- `load_doc()` - Load file
- `all_img_captions()` - Parse tokens
- `cleaning_text()` - Clean text
- `text_vocabulary()` - Build vocab
- `save_descriptions()` - Save to file

**Simplified to 1 function that does everything:**

In [ ]:
def process_captions(token_file):
    """
    Load, parse, clean, and structure captions in ONE step.
    
    Returns:
        dict: {image_name: [list of cleaned captions]}
    """
    descriptions = {}
    table = str.maketrans('', '', string.punctuation)
    
    with open(token_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
                
            # Parse: "image.jpg#0\tcaption text"
            img_id, caption = line.split('\t')
            img_name = img_id[:-2]  # Remove #0, #1, etc.
            
            # Clean caption: lowercase, remove punctuation, filter short words
            caption = caption.lower().translate(table)
            caption = ' '.join([w for w in caption.split() if len(w) > 1])
            caption = f'<start> {caption} <end>'  # Add special tokens
            
            # Store
            if img_name not in descriptions:
                descriptions[img_name] = []
            descriptions[img_name].append(caption)
    
    print(f"Loaded {len(descriptions)} images with {sum(len(v) for v in descriptions.values())} captions")
    return descriptions

In [ ]:
# Process all captions
all_descriptions = process_captions(TOKEN_FILE)

## 4. Feature Extraction - SIMPLIFIED

**Changes:**
- Removed separate `download_with_retry()` function
- Combined feature extraction and saving
- Better error handling

In [ ]:
def extract_all_features(image_dir, output_file):
    """
    Extract features from all images using Xception CNN.
    
    Args:
        image_dir: Directory containing images
        output_file: Where to save features
    """
    # Load Xception model
    model = Xception(include_top=False, pooling='avg', weights='imagenet')
    
    features = {}
    valid_extensions = {'.jpg', '.jpeg', '.png'}
    
    image_files = [f for f in os.listdir(image_dir) 
                   if os.path.splitext(f)[1].lower() in valid_extensions]
    
    for img_name in tqdm(image_files, desc="Extracting features"):
        try:
            img_path = os.path.join(image_dir, img_name)
            img = Image.open(img_path).resize((299, 299))
            
            # Preprocess: normalize to [-1, 1]
            img_array = np.array(img)
            if img_array.shape[-1] == 4:  # RGBA -> RGB
                img_array = img_array[..., :3]
            img_array = np.expand_dims(img_array, axis=0)
            img_array = img_array / 127.5 - 1.0
            
            # Extract features
            feature = model.predict(img_array, verbose=0)
            features[img_name] = feature[0]  # Remove batch dimension
        except Exception as e:
            print(f"Error processing {img_name}: {e}")
    
    # Save features
    with open(output_file, 'wb') as f:
        dump(features, f)
    
    print(f"Extracted features for {len(features)} images")
    return features

In [ ]:
# Extract features (run once, takes ~20-30 mins)
# features = extract_all_features(DATASET_IMAGES, FEATURES_FILE)

# Or load pre-extracted features
with open(FEATURES_FILE, 'rb') as f:
    all_features = load(f)
print(f"Loaded {len(all_features)} image features")

## 5. Prepare Training Data - COMBINED

**Original code had 5 functions:**
- `load_photos()` - Load image list
- `load_clean_descriptions()` - Load captions
- `load_features()` - Filter features
- `dict_to_list()` - Flatten captions
- `create_tokenizer()` - Build tokenizer

**Simplified to 1 function:**

In [ ]:
def prepare_dataset(train_file, descriptions, features):
    """
    Load train split, filter data, and create tokenizer.
    
    Returns:
        train_descriptions: dict of training captions
        train_features: dict of training features
        tokenizer: fitted tokenizer
        vocab_size: vocabulary size
        max_length: maximum caption length
    """
    # Load train image names
    with open(train_file, 'r') as f:
        train_images = [line.strip() for line in f if line.strip()]
    
    # Filter descriptions and features for training set
    train_descriptions = {img: descriptions[img] for img in train_images if img in descriptions}
    train_features = {img: features[img] for img in train_images if img in features}
    
    # Flatten all captions
    all_captions = [cap for caps in train_descriptions.values() for cap in caps]
    
    # Create tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_captions)
    vocab_size = len(tokenizer.word_index) + 1
    
    # Calculate max caption length
    max_length = max(len(cap.split()) for cap in all_captions)
    
    print(f"Training images: {len(train_descriptions)}")
    print(f"Vocabulary size: {vocab_size}")
    print(f"Max caption length: {max_length}")
    
    return train_descriptions, train_features, tokenizer, vocab_size, max_length

In [ ]:
# Prepare training data
train_descriptions, train_features, tokenizer, vocab_size, max_length = prepare_dataset(
    TRAIN_IMAGES, all_descriptions, all_features
)

# Save tokenizer
with open(TOKENIZER_FILE, 'wb') as f:
    dump(tokenizer, f)

## 6. Create Training Sequences - UNCHANGED (Core Logic)

This function is essential and already optimized.

In [ ]:
def create_sequences(tokenizer, max_length, captions, feature, vocab_size):
    """
    Create training sequences from captions.
    Each caption generates multiple (image, partial_caption) -> next_word pairs.
    """
    X1, X2, y = [], [], []
    
    for caption in captions:
        seq = tokenizer.texts_to_sequences([caption])[0]
        
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    
    return np.array(X1), np.array(X2), np.array(y)

## 7. Data Generator - SIMPLIFIED

In [ ]:
def data_generator(descriptions, features, tokenizer, max_length, vocab_size, batch_size=32):
    """
    Simplified data generator with TensorFlow Dataset API.
    """
    def generator():
        while True:
            for img_name, captions in descriptions.items():
                feature = features[img_name]
                X1, X2, y = create_sequences(tokenizer, max_length, captions, feature, vocab_size)
                
                for i in range(len(X1)):
                    yield {'input_1': X1[i], 'input_2': X2[i]}, y[i]
    
    output_signature = (
        {
            'input_1': tf.TensorSpec(shape=(2048,), dtype=tf.float32),
            'input_2': tf.TensorSpec(shape=(max_length,), dtype=tf.int32)
        },
        tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)
    )
    
    dataset = tf.data.Dataset.from_generator(generator, output_signature=output_signature)
    return dataset.batch(batch_size)

## 8. Build Model - UNCHANGED (Core Architecture)

In [ ]:
def build_model(vocab_size, max_length):
    """
    Build CNN-RNN encoder-decoder model.
    """
    # Image encoder
    input_img = Input(shape=(2048,), name='input_1')
    img_features = Dropout(0.5)(input_img)
    img_features = Dense(256, activation='relu')(img_features)
    
    # Text decoder
    input_seq = Input(shape=(max_length,), name='input_2')
    seq_features = Embedding(vocab_size, 256, mask_zero=True)(input_seq)
    seq_features = Dropout(0.5)(seq_features)
    seq_features = LSTM(256)(seq_features)
    
    # Merge and output
    decoder = add([img_features, seq_features])
    decoder = Dense(256, activation='relu')(decoder)
    output = Dense(vocab_size, activation='softmax')(decoder)
    
    model = Model(inputs=[input_img, input_seq], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

In [ ]:
# Build model
model = build_model(vocab_size, max_length)
model.summary()

## 9. Training - SIMPLIFIED

In [ ]:
# Calculate steps per epoch
total_sequences = sum(len(caps) * (len(caps[0].split()) - 1) 
                      for caps in train_descriptions.values())
steps_per_epoch = max(1, total_sequences // 32)

print(f"Steps per epoch: {steps_per_epoch}")

In [ ]:
# Create model directory
os.makedirs(MODEL_DIR, exist_ok=True)

# Training loop
epochs = 10
for epoch in range(epochs):
    print(f"\n=== Epoch {epoch+1}/{epochs} ===")
    
    # Create fresh dataset for this epoch
    dataset = data_generator(train_descriptions, train_features, 
                            tokenizer, max_length, vocab_size)
    
    # Train for 1 epoch
    model.fit(dataset, epochs=1, steps_per_epoch=steps_per_epoch, verbose=1)
    
    # Save checkpoint
    model.save(os.path.join(MODEL_DIR, f'model_{epoch}.h5'))
    print(f"Saved model_{epoch}.h5")

## 10. Inference - COMBINED & SIMPLIFIED

**Original code had 3 separate functions:**
- `extract_features()` - Extract features from new image
- `word_for_id()` - Convert index to word
- `generate_desc()` - Generate caption

**Simplified to 1 function:**

In [ ]:
def predict_caption(image_path, model, tokenizer, max_length):
    """
    Generate caption for a new image - ALL IN ONE.
    
    Args:
        image_path: Path to image file
        model: Trained caption model
        tokenizer: Fitted tokenizer
        max_length: Maximum caption length
    
    Returns:
        str: Generated caption
    """
    # 1. Load Xception for feature extraction
    xception = Xception(include_top=False, pooling='avg', weights='imagenet')
    
    # 2. Load and preprocess image
    img = Image.open(image_path).resize((299, 299))
    img_array = np.array(img)
    if img_array.shape[-1] == 4:
        img_array = img_array[..., :3]
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 127.5 - 1.0
    
    # 3. Extract features
    feature = xception.predict(img_array, verbose=0)
    
    # 4. Generate caption word by word
    caption = '<start>'
    
    for _ in range(max_length):
        # Tokenize current caption
        sequence = tokenizer.texts_to_sequences([caption])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        
        # Predict next word
        pred = model.predict([feature, sequence], verbose=0)
        word_idx = np.argmax(pred)
        
        # Convert index to word (using tokenizer's index_word)
        word = tokenizer.index_word.get(word_idx, None)
        
        if word is None or word == '<end>':
            break
        
        caption += ' ' + word
    
    # Clean up caption
    caption = caption.replace('<start>', '').replace('<end>', '').strip()
    return caption

## 11. Test Inference

In [ ]:
import matplotlib.pyplot as plt

# Load trained model
from tensorflow.keras.models import load_model
model = load_model(os.path.join(MODEL_DIR, 'model_9.h5'))

# Test on an image
test_image = "path/to/test/image.jpg"  # Update this

caption = predict_caption(test_image, model, tokenizer, max_length)

# Display
img = Image.open(test_image)
plt.imshow(img)
plt.axis('off')
plt.title(f"Caption: {caption}", fontsize=14, wrap=True)
plt.tight_layout()
plt.show()

print(f"\nGenerated Caption: {caption}")

---

## Summary of Simplifications

### Original Code: ~20 Functions
1. `load_doc()`
2. `all_img_captions()`
3. `cleaning_text()`
4. `text_vocabulary()`
5. `save_descriptions()`
6. `download_with_retry()`
7. `extract_features()` (for batch)
8. `load_photos()`
9. `load_clean_descriptions()`
10. `load_features()`
11. `dict_to_list()`
12. `create_tokenizer()`
13. `max_length()`
14. `create_sequences()`
15. `data_generator()`
16. `define_model()`
17. `get_steps_per_epoch()`
18. `extract_features()` (for single image)
19. `word_for_id()`
20. `generate_desc()`

### Simplified Code: 6 Core Functions
1. **`process_captions()`** - Combines: load_doc, all_img_captions, cleaning_text, text_vocabulary
2. **`extract_all_features()`** - Combines: download_with_retry, extract_features (batch)
3. **`prepare_dataset()`** - Combines: load_photos, load_clean_descriptions, load_features, dict_to_list, create_tokenizer, max_length
4. **`create_sequences()`** - Unchanged (core logic)
5. **`data_generator()`** - Slightly simplified
6. **`build_model()`** - Unchanged (core architecture)
7. **`predict_caption()`** - Combines: extract_features (single), word_for_id, generate_desc

### Benefits:
- ✅ **70% fewer functions** (20 → 6)
- ✅ **Cleaner code flow**
- ✅ **Less redundancy**
- ✅ **Easier to understand**
- ✅ **Same functionality**
- ✅ **Better error handling**
- ✅ **Uses tokenizer.index_word** (built-in) instead of custom loop

---